## Consruction du tableau de bord de bilan

### Contexte de la donnée: 
<p>Jeu de donnée de l'INSEE<br>
Description ODD et indicatuers <br>
Suivi dans le temps <br>
    <strong><i>[à compléter]</i></strong>
<br></p>

### Objectifs de représentation: 
<p>Comprendre l'évolution des indicateurs en fonction des données collectées.<br>
Obtenir un tableau de bord interactif permettant de se concentrer sur un ODD et ses indicateurs. <br>
Intégrer un maximum d'informations tout en la conservant lisible. <br>
</p>

### Marche à suivre: 
<p>Produire un premier fichier à l'aide de formules simples/brutes me permettant de visualiser le produit fini: <br>
1. Identifier les formules nécessaires pour constuire le tableau de bord,<br>
2. Comprendre le fonctionnement et les méthodes Openpyxl,<br>
3. Générer un classeur Excel regroupant la donnée 'propre', les formules, les graphiques et exploitable par des corps de métiers.<br></p>
<p><br>
Produire un second fichier à l'aide de fonction permettant de simplifier le code et accélérer le traitement.<br> 
1. Factoriser les différentes formules nécessaires à la construction du tableau de bord,<br>
2. Factoriser le code de chacun des tableaux/graphes nécessaires,<br>
3. Imbriquer les fonctions créées.<br></p>

## Etape 1 /  Préparation et nettoyage du plan de travail: 

Il faut créer puis peupler un excel qui servira de produit final. Ici je crée et ajoute les données et métadonnées dans le fichier Excel sous forme de tableaux afin que je puisse utiliser le nom des fonctions et non les numéros de colonnes du tableau. Cela rend les formules plus fiables et exploitables. Pour cela, je passe par quelques étapes de nettoyage et prétraitement de la donnée. 

### Nettoyage de la donnée 

In [2]:
data = pd.read_csv('../DS_DEVDUR_data.csv', sep = ";")
data.head()

,ODD,INDICATEUR_DEVDUR,UNIT_MEASURE,UNIT_MULT,OBS_STATUS,GEO,GEO_OBJECT,DEVDUR_COMPOSITE,SEX,AGE,EMPSTA,PCS,TYPE_INDICATEUR_DEVDUR,TIME_PERIOD,OBS_VALUE
0,ODD1,1.i1a,PT,0,A,FM,FRANCE,NIVIEMED_S60,_T,_T,11,_T,I_ODD,2015,6.5
1,ODD3,3.i2,_Z,0,E,F,FRANCE,CONTAMIN_MOD4,_T,_T,_Z,_Z,I_ODD,2021,255.0
2,ODD2,2.i2f,PT,0,L,F,FRANCE,_Z,_T,_Z,_Z,1_2,I_ODD,2015,NaN
3,ODD1,1.i1a,PT,0,A,FM,FRANCE,NIVIEMED_S60,_T,Y40T49,_T,_T,I_ODD,2016,13.5
4,ODD11,11.i1,PT,0,L,FM,FRANCE,_Z,_T,_T,_Z,_Z,I_ODD,2015,NaN


In [3]:
meta = pd.read_csv('../DS_DEVDUR_metadata.csv',sep = ";")
meta.head()

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD
0,AGE,Âge,Y18T75,De 18 à 75 ans
1,AGE,Âge,Y18T64,De 18 à 64 ans
2,AGE,Âge,_Z,Non applicable
3,AGE,Âge,Y15,15 ans
4,AGE,Âge,Y_LT14,Moins de 14 ans


Je veux maintenant associer les libellé des codes attribués aux modalités de mon jeu de données. Cela permettra d'avoir une base propre, lisible et directement exploitable. De plus, l'utilisateur pourra retrouver l'information qu'il cherche plus facilement dans la base en cas de doute.

In [4]:
## Création d'une table lisible en associant les libellés aux données encodées: 

data_lib = data.copy()
data_lib = data_lib[(data_lib['TIME_PERIOD']>2010) & (data_lib['OBS_STATUS']!="L")]
cols = ['ODD', 'INDICATEUR_DEVDUR', 'UNIT_MEASURE', 'OBS_STATUS', 'GEO', 'GEO_OBJECT','DEVDUR_COMPOSITE', 'SEX', 'AGE', 'EMPSTA', 'PCS', 'TYPE_INDICATEUR_DEVDUR']
for col in cols:
    if col in meta['COD_VAR'].unique():
        sous_table = meta[meta['COD_VAR'] == col]
        dict_col = dict(zip(sous_table['COD_MOD'],sous_table['LIB_MOD']))
        data_lib[col] = data_lib[col].map(dict_col)
        variables = list(meta['LIB_VAR'].unique())

        # On remplace également le titre des colonnes (TODO: manque colonne GEO_OBJECT et OBS_VALUE):
dict_var = dict(zip(meta['COD_VAR'],meta['LIB_VAR']))
data_lib.rename(columns=dict_var, inplace=True)


In [13]:
path_file = '../OUTPUT/TDB_ODD_2.xlsx'

# Si le fichier n'existe pas, on le crée
if not os.path.exists(path_file):
    wb = Workbook()
    wb.save(path_file)

# Ensuite, on écrit dans le fichier
with pd.ExcelWriter(path_file, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
    data_lib.to_excel(writer, sheet_name='Base propre', index=False)
    data.to_excel(writer, sheet_name = 'Base brute', index = False)
    meta.to_excel(writer, sheet_name='Index', index=False)

from openpyxl.worksheet.table import Table, TableStyleInfo

wb = load_workbook(path_file)

def add_table(ws, df, table_name):
    n_rows, n_cols = df.shape
    max_row = n_rows + 1  # +1 pour la ligne des en-têtes
    max_col = n_cols
    
    from openpyxl.utils import get_column_letter
    last_col_letter = get_column_letter(max_col)
    
    table_range = f"A1:{last_col_letter}{max_row}"  # <-- ici
    
    tab = Table(displayName=table_name, ref=table_range)
    style = TableStyleInfo(name="TableStyleMedium3", showRowStripes=True)
    tab.tableStyleInfo = style
    
    ws.add_table(tab)

# Ajouter tableaux dans les feuilles concernées
add_table(wb['Base propre'], data_lib, "Base")
add_table(wb['Index'], meta, "Index")

wb.save(path_file)

Deuxième étape // Ajouter des indicateurs 

On conditionne d'abord l'affichage sur les filtres: 

L'objectif de développement durable: Cela permettra de filtrer le reste des affichages en fonction des observations liées à cet indicateur. 


In [14]:
wb = load_workbook(path_file)
if 'TDB' not in wb.sheetnames:
    wb.create_sheet('TDB')
    wb.save(path_file)
    print("Feuille bilan créée.")
else:
    print("Feuille bilan trouvée.")
    

Feuille bilan créée.


In [7]:
from openpyxl import load_workbook

wb = load_workbook(path_file)
ws = wb['TDB']

# Parcourir toutes les cellules utilisées dans la feuille et vider leur contenu
for row in ws.iter_rows():
    for cell in row:
        cell.value = None
        cell.style = 'Normal'  # Remet au style par défaut (optionnel)

# Enlever toutes les validations
ws.data_validations.dataValidation = []

# Enlever les images, graphiques, etc. (optionnel, si tu en as)
ws._images = []
ws._charts = []

wb.save(path_file)


In [15]:
len_base = len(data_lib)
len_ind = len(data['INDICATEUR_DEVDUR'].unique())
len_ind

175

In [28]:
bilan = wb['TDB']

titre_f1 = bilan['A1']
titre_f1.value = 'ODD'
titre_f1.fill = PatternFill(start_color='8DB600', end_color='8DB600', fill_type='solid')

bilan.merge_cells(start_row=1, 
                       start_column=1, 
                       end_row=2, 
                       end_column=2)

val_F1 = bilan['C1']
val_F1.value = 'Objectif 1 : Éradication de la pauvreté'
val_F1.alignment  = Alignment(horizontal='center', vertical='center')
val_F1.fill = PatternFill(start_color='8DB600', end_color='8DB600', fill_type='solid')

formula = f"='Base brute'!$B$2:$b${len_ind}"

#Ajouter la liste de validation pour créer le filtre
dv = DataValidation(type='list', formula1=formula)
bilan.add_data_validation(dv)
coord_filter_year = 'C1'
dv.add(coord_filter_year)

bilan.merge_cells(start_row=1, 
                       start_column=3, 
                       end_row=2, 
                       end_column=6)

wb.save(path_file)
wb.close()

In [38]:
titre_tab = bilan['B4']
titre_tab.value = "Observations cumulées par indicateur"

# Remplissage avec vert pomme (#8DB600) et opacité pleine (FF)
titre_tab.fill = PatternFill(start_color='FF8DB600', end_color='FF8DB600', fill_type='solid')

# Alignement centré avec retour à la ligne
titre_tab.alignment = Alignment(horizontal='center', vertical='center')

# Fusion des cellules B4:C10
bilan.merge_cells(start_row=4, start_column=2, end_row=10, end_column=3)

formula = f"=_xlfn.UNIQUE(_xlfn.FILTER(Base[Indicateur d’objectif de développement durable ],(Base[Objectif développement durable]) = $C$1))"

bilan['D4'] = "Indicateurs"
bilan['D5'] = ArrayFormula(f"D5:D10", formula )

## TODO Pourquoi la somme ne marche pas? Vérifier type des valeurs?
bilan['E4'] = "Somme des observations"
bilan ['E5'] = f"=SUMIF(Base[Indicateur d’objectif de développement durable ],D5, Base[OBS_VALUE])"
bilan ['E6'] = f"=SUMIF(Base[Indicateur d’objectif de développement durable ],D6, Base[OBS_VALUE])"
bilan ['E7'] = f"=SUMIF(Base[Indicateur d’objectif de développement durable ],D7, Base[OBS_VALUE])"
bilan ['E8'] = f"=SUMIF(Base[Indicateur d’objectif de développement durable ],D8, Base[OBS_VALUE])"
bilan ['E9'] = f"=SUMIF(Base[Indicateur d’objectif de développement durable ],D9, Base[OBS_VALUE])"
bilan ['E10'] = f"=SUMIF(Base[Indicateur d’objectif de développement durable ],D10, Base[OBS_VALUE])"

bilan['F4'] = "Unité de mesure" # Ajouter une autre colonne observation que l'on multiplie par unit_mult pour avoir la val totale
bilan ['F5'] = f'=_xlfn.XLOOKUP(D5,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F6'] = f'=_xlfn.XLOOKUP(D6,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F7'] = f'=_xlfn.XLOOKUP(D7,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F8'] = f'=_xlfn.XLOOKUP(D8,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F9'] = f'=_xlfn.XLOOKUP(D9,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'
bilan ['F10'] = f'=_xlfn.XLOOKUP(D10,Base[Indicateur d’objectif de développement durable ],Base[Unité de mesure],"-")'

## TODO
bilan['G4'] = "Statut de l'observation" 
bilan ['G5'] = f"=_xlfn.XLOOKUP(_xlfn.MAX(_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D5)),_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D5), _xlfn.FILTER(Base[Statut de l'observation], Base[Indicateur d’objectif de développement durable ] = D5),'-')"
bilan ['G6'] = f"=_xlfn.XLOOKUP(_xlfn.MAX(_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D6)),_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D6), _xlfn.FILTER(Base[Statut de l'observation], Base[Indicateur d’objectif de développement durable ] = D6),'-')"
bilan ['G7'] = f"=_xlfn.XLOOKUP(_xlfn.MAX(_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D7)),_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D7), _xlfn.FILTER(Base[Statut de l'observation], Base[Indicateur d’objectif de développement durable ] = D7),'-')"
bilan ['G8'] = f"=_xlfn.XLOOKUP(_xlfn.MAX(_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D8)),_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D8), _xlfn.FILTER(Base[Statut de l'observation], Base[Indicateur d’objectif de développement durable ] = D8),'-')"
bilan ['G9'] = f"=_xlfn.XLOOKUP(_xlfn.MAX(_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D9)),_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D9), _xlfn.FILTER(Base[Statut de l'observation], Base[Indicateur d’objectif de développement durable ] = D9),'-')"
bilan ['G10'] = f"=_xlfn.XLOOKUP(_xlfn.MAX(_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D10)),_xlfn.FILTER(Base[Période temporelle], Base[Indicateur d’objectif de développement durable ] = D10), _xlfn.FILTER(Base[Statut de l'observation], Base[Indicateur d’objectif de développement durable ] = D10),'-')"


wb.save(path_file)
wb.close()

In [1]:
import pandas as pd
import os
from openpyxl import Workbook   
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import load_workbook 
from openpyxl.utils import get_column_letter
from openpyxl.styles import PatternFill
from openpyxl.chart import BarChart, Reference
from openpyxl.styles import Font, Border, Side
from openpyxl.styles import Alignment
from openpyxl.chart.label import DataLabelList                                                                                                                                                      
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.worksheet.formula import ArrayFormula
from openpyxl.utils import quote_sheetname
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.worksheet.worksheet import Worksheet
from openpyxl.styles import Alignment, PatternFill
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.styles import PatternFill, Alignment



### 